In [1]:
import sys
!{sys.executable} -m pip install opendartreader finance-datareader plotly tqdm --quiet
import FinanceDataReader as fdr
import pandas as pd
from tqdm import tqdm
import OpenDartReader

dart = OpenDartReader('d74227e34998d564a9671c471b27b9c7701d3d03')  

kospi = fdr.StockListing('KOSPI')
kospi_top50 = kospi.sort_values(by='Marcap', ascending=False).head(50)
df = kospi_top50[['Code', 'Name']].copy()

corp_codes = []
for code, name in tqdm(zip(df['Code'], df['Name']), total=len(df)):
    try:
        corp = dart.company(code)
        corp_code = corp['corp_code']
    except:
        corp_code = ''
    corp_codes.append(corp_code)

df['DART_code'] = corp_codes

records = []

for i, row in tqdm(df.iterrows(), total=len(df)):
    code = row['Code']
    name = row['Name']
    dart_code = row['DART_code']

    for year in range(2015, 2025):
        try:
            if dart_code == '':
                if name.endswith("우"):
                    target_name = name[:-1]
                    preferred = True
                else:
                    target_name = name
                    preferred = False

                report_df = dart.report(year=year, name=target_name)
                if report_df is None or report_df.empty:
                    continue
                if preferred:
                    report_df = report_df[report_df['stock_knd'].str.contains('우', na=False)]
                else:
                    report_df = report_df[report_df['stock_knd'].str.contains('보통주', na=False)]
            else:
                report_df = dart.report(year=year, corp_code=dart_code)
                if report_df is None or report_df.empty:
                    continue
                report_df = report_df[report_df['stock_knd'].str.contains('보통주', na=False)]

            report_df = report_df.dropna(subset=['cash_dvdn', 'eps'])
            if len(report_df) > 0:
                cash_dvdn = int(report_df.iloc[0]['cash_dvdn'])
                eps = int(report_df.iloc[0]['eps'])
                payout_ratio = cash_dvdn / eps if eps != 0 else 0
                records.append({
                    'Code': code,
                    'Name': name,
                    'DART_code': dart_code,
                    '회계연도': year,
                    '주당현금배당금액': cash_dvdn,
                    '주당순이익': eps,
                    '배당성향': round(payout_ratio, 6)
                })
        except Exception as e:
            continue

result_df = pd.DataFrame(records)

result_df.to_excel('숙제6.xlsx', index=False)

100%|████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 8223.80it/s]
